In [1]:
# Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import csv
import mysql.connector
from mysql.connector import Error
import config
from rdkit.DataStructs import TanimotoSimilarity
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import gc
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import vstack, hstack, csr_matrix
from rdkit.Chem import Draw
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from scipy.sparse import vstack
from sklearn.metrics import average_precision_score

In [29]:
train_df = pd.read_csv('train_data_strings.csv')
test_df = pd.read_csv('test_data_strings.csv')

In [30]:
train_df['molVector'] = train_df['molVector'].apply(lambda x: [int(i) for i in x.split(',')])
test_df['molVector'] = test_df['molVector'].apply(lambda x: [int(i) for i in x.split(',')])

In [31]:
first_row_vector = train_df['molVector'].iloc[0]
max_len = len(first_row_vector)
print(max_len)

512


In [32]:
"""about 6 minutes for this code"""
molVector_train_df = pd.DataFrame(train_df['molVector'].to_list(), columns=[f"molVect{i + 1}" for i in range(max_len)])
molVector_test_df = pd.DataFrame(test_df['molVector'].to_list(), columns=[f"molVect{i + 1}" for i in range(max_len)])

In [33]:
print(molVector_train_df.dtypes)

molVect1      int64
molVect2      int64
molVect3      int64
molVect4      int64
molVect5      int64
              ...  
molVect508    int64
molVect509    int64
molVect510    int64
molVect511    int64
molVect512    int64
Length: 512, dtype: object


In [34]:
"""Takes around 12 minutes for this code"""
for col in molVector_train_df.columns:
    molVector_train_df[col] = molVector_train_df[col].astype(np.int8)
print('Changed type to int8...')
train_df = pd.concat([train_df.drop(['molVector'], axis=1), molVector_train_df], axis=1)
print('combined training set...')

for col in molVector_test_df.columns:
    molVector_test_df[col] = molVector_test_df[col].astype(np.int8)
print('Changed type to int8...')
test_df = pd.concat([test_df.drop(['molVector'], axis=1), molVector_test_df], axis=1)
print('combined testing set...')


Changed type to int8...
combined training set...
Changed type to int8...
combined testing set...


In [35]:
# Drop 'id' column from training and testing data
train_df = train_df.drop(columns=['id'])
test_df = test_df.drop(columns=['id'])

# Define target variable for training data
y_train = train_df['binds']
X_train = train_df.drop(columns=['binds'])

# Define target variable for testing data (if applicable)
y_test = test_df['binds']
X_test = test_df.drop(columns=['binds'])


In [36]:
del train_df, test_df
gc.collect()

NameError: name 'SMOTE' is not defined

# XGBoost Training

In [38]:
# Create XGBoost specific DMatrix data format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [11]:
# params = {
#     'objective': 'binary:logistic',
#     'n_estimators': 10,
#     'seed': 42,
#     'eta': 0.01,
#     'alpha': 10,
#     'max_depth': 5,
# }
# 
# # Perform cross-validation
# cv_results = xgb.cv(
#     params,
#     dtrain,
#     num_boost_round=50,
#     seed=42,
#     nfold=5,
#     metrics={"error", "logloss"},
#     early_stopping_rounds=10
# )
# 
# # Print cross-validation results
# print(cv_results)


In [52]:
# Set parameters
params = {
    'objective': 'binary:logistic',
    'n_estimators': 1000,  
    'learning_rate':0.005,
    'alpha': 10,  
    'lambda': 1,  
    'max_depth': 3,
    'min_child_weight': 50,
    'n_jobs': -1
}

# Train model
model = xgb.train(params, dtrain)

[22:00:48] WARNING: C:\b\abs_7diruzi3as\croot\xgboost-split_1712794727514\work\src\learner.cc:767: 
Parameters: { "n_estimators" } are not used.



In [13]:
model.save_model('xgb_model.json')

# Neural Network

In [51]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("CUDA Available: ", cuda_available)

# Get the number of GPUs available
if cuda_available:
    num_gpus = torch.cuda.device_count()
    print("Number of GPUs Available: ", num_gpus)
    for i in range(num_gpus):
        print("GPU ", i, ": ", torch.cuda.get_device_name(i))
else:
    print("No GPU available, using CPU instead.")


AttributeError: module 'torch' has no attribute 'cuda'

In [50]:
print("Torch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())

AttributeError: module 'torch' has no attribute '__version__'

# Model Results

In [53]:
# Make the prediction
raw_predictions_train = model.predict(dtrain)

# Create a dataframe with the predicted probabilities
probabilities_train = pd.DataFrame(raw_predictions_train)
print(probabilities_train.head())

# Convert probabilities into binary output
predictions_train = [1 if proba > 0.5 else 0 for proba in raw_predictions_train]

# Now print the accuracy with these classes
print("Accuracy:", accuracy_score(y_train, predictions_train))
print("Precision:", precision_score(y_train, predictions_train))
print("Recall:", recall_score(y_train, predictions_train))
print("F1 Score:", f1_score(y_train, predictions_train))

print(classification_report(y_train, predictions_train, target_names=['Class 0', 'Class 1']))

conf_matrix_train = confusion_matrix(y_train, predictions_train)
print("Confusion Matrix:\n", conf_matrix_train)

fpr_train, tpr_train, thresholds_train = roc_curve(y_train, probabilities_train)
roc_auc_train = auc(fpr_train, tpr_train)

print("AUC Score (Training):", roc_auc_train)

# Average Precision Score
average_precision_train = average_precision_score(y_train, probabilities_train)
print('Mean Average Precision (Training): {0:0.2f}'.format(average_precision_train))


          0
0  0.475814
1  0.524442
2  0.524442
3  0.475814
4  0.524442
Accuracy: 0.9973715
Precision: 1.0
Recall: 0.994743
F1 Score: 0.997364572779551
              precision    recall  f1-score   support

     Class 0       0.99      1.00      1.00   1000000
     Class 1       1.00      0.99      1.00   1000000

    accuracy                           1.00   2000000
   macro avg       1.00      1.00      1.00   2000000
weighted avg       1.00      1.00      1.00   2000000

Confusion Matrix:
 [[1000000       0]
 [   5257  994743]]
AUC Score (Training): 0.9973719999999999
Mean Average Precision (Training): 1.00


In [54]:
# Make the prediction
raw_predictions = model.predict(dtest)

# Create a dataframe with the predicted probabilities  
probabilities = pd.DataFrame(raw_predictions)
print(probabilities.head())

# Convert probabilities into binary output
predictions = [1 if proba > 0.5 else 0 for proba in raw_predictions]

# Now print the accuracy with these classes
print("Accuracy:", accuracy_score(y_test, predictions))
print("Precision:", precision_score(y_test, predictions))
print("Recall:", recall_score(y_test, predictions))
print("F1 Score:", f1_score(y_test, predictions))

print(classification_report(y_test, predictions, target_names=['Class 0', 'Class 1']))

conf_matrix = confusion_matrix(y_test, predictions)
print("Confusion Matrix:\n", conf_matrix)

fpr, tpr, thresholds = roc_curve(y_test, probabilities)
roc_auc = auc(fpr, tpr)

print("AUC Score:", roc_auc)

# Average Precision Score
average_precision = average_precision_score(y_test, probabilities)
print('Mean Average Precision (micro): {0:0.2f}'.format(average_precision))

          0
0  0.524442
1  0.524442
2  0.524442
3  0.524442
4  0.524412
Accuracy: 0.5733727068380386
Precision: 0.5396096777086548
Recall: 0.9995694229250084
F1 Score: 0.7008636407938935
              precision    recall  f1-score   support

     Class 0       1.00      0.15      0.26    589906
     Class 1       0.54      1.00      0.70    589906

    accuracy                           0.57   1179812
   macro avg       0.77      0.57      0.48   1179812
weighted avg       0.77      0.57      0.48   1179812

Confusion Matrix:
 [[ 86820 503086]
 [   254 589652]]
AUC Score: 0.9889508116939492
Mean Average Precision (micro): 0.99


# Code for Creating Submission Using Model

In [16]:
def connect_to_database():
    """Connect to the MySQL database using settings from the config module."""
    try:
        print("Connecting to the MySQL database...")
        conn = mysql.connector.connect(**config.DATABASE_CONFIG)
        if conn.is_connected():
            print("Connection established.")
        else:
            print("Connection failed.")
        return conn
    except Error as e:
        print(f"The error '{e}' occurred")
        return None

def close_connection(conn):
    """Close the database connection."""
    if conn.is_connected():
        conn.close()
        print("The connection is closed.")

def fetch_protein_mapping(cursor):
    """Fetch protein name to numeric mapping from the database."""
    query = "SELECT protein_name, protein_numeric FROM protein_mapping"
    cursor.execute(query)
    mapping_data = cursor.fetchall()
    print('fetched protein mapping...')
    return {name: num for name, num in mapping_data}

In [17]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm
import gc

# Load CSV data
test_submission_data = pd.read_csv('leash-BELKA/test.csv')

# Function to convert SMILES string to a fingerprint
def smiles_to_fingerprint(smiles, radius=2, nBits=512):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        return list(AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits))
    else:
        return [0]*nBits  # Return a zero vector if the molecule parsing fails

# Process SMILES strings in batches
def process_in_batches(smiles_series, batch_size=1000):
    num_batches = (len(smiles_series) + batch_size - 1) // batch_size
    results = []
    for i in tqdm(range(num_batches), desc="Processing Batches"):
        batch = smiles_series.iloc[i*batch_size:(i+1)*batch_size].apply(smiles_to_fingerprint)
        results.extend(batch)
    return pd.Series(results)

# Create dataframe and process fingerprints
test_submission_df = pd.DataFrame()
test_submission_df['id'] = test_submission_data['id']
test_submission_df['molecule_smiles'] = process_in_batches(test_submission_data['molecule_smiles'], batch_size=500)

Processing Batches: 100%|██████████| 3350/3350 [19:59<00:00,  2.79it/s]


In [18]:
# Assuming you have a way to fetch protein mappings similar to the database method shown

conn = connect_to_database()
cursor = conn.cursor()
protein_mapping = fetch_protein_mapping(cursor)
cursor.close()
conn.close()

Connecting to the MySQL database...
Connection established.
fetched protein mapping...


In [19]:
test_submission_df['Protein_numeric'] = test_data['protein_name'].map(protein_mapping)

In [20]:
# Adjust based on your fingerprint length
"""Expect about 9 minutes for this code"""
molVector_test_df = pd.DataFrame(test_submission_df['molecule_smiles'].tolist(), columns=[f"molVect{i + 1}" for i in range(max_len)])
print('converted to many columns...')
for col in molVector_test_df.columns:
    molVector_test_df[col] = molVector_test_df[col].astype(np.int8)
print('Converted type to int8...')
test_submission_df = pd.concat([test_submission_df.drop(['molecule_smiles'], axis=1), molVector_test_df], axis=1)

Converted type to int8 and reshaped the dataframe...


In [ ]:
# To load model
model = xgb.Booster()
model.load_model('xgb_model.json')

In [24]:

# Prepare data for prediction
dtest1 = xgb.DMatrix(test_submission_df.drop(['id'], axis=1))

In [25]:
raw_predictions = model.predict(dtest1)

# Save predictions
test_submission_df['binds'] = raw_predictions

C:\Users\funkb\AppData\Local\Temp\ipykernel_2428\2523094138.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df['binds'] = raw_predictions


In [26]:
test_submission_df[['id', 'binds']].to_csv('Submission2XGBoost.csv', index=False)